In [22]:
from google.colab import drive

drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [23]:
file_path = './drive/MyDrive/LGAImers/'

In [24]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn import metrics

from sklearn.model_selection import train_test_split

In [25]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [26]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score, all_nrmse

In [27]:
df = pd.read_csv(file_path + './IQR_train.csv')

# NaN 값 0으로 변환
df1 = pd.DataFrame(df)
df1 = df.fillna(0)
print(df1)

       Unnamed: 0    X_01   X_03    X_06   X_07    X_08    X_09  X_12  X_13  \
0               0  70.544  67.47  74.983  29.45    0.00  245.71  4.34  0.18   
1               1  69.524  65.17  72.943  28.73    0.00  233.61  4.38  0.18   
2               2  72.583  64.07  72.943  28.81  105.77  272.20  4.36  0.15   
3               3  71.563  67.57   0.000  28.92  115.21  255.36  4.33  0.00   
4               4  69.524  63.57  70.904  29.68  103.38  241.46  4.35  0.16   
...           ...     ...    ...     ...    ...     ...     ...   ...   ...   
39602       39602  66.465  62.27  66.825  30.20   77.83  298.05  4.36  0.15   
39603       39603  66.465  62.77  66.825  29.21  102.25  270.67  4.40  0.13   
39604       39604  68.504  64.67  68.864  29.96  102.61  198.07  4.38  0.14   
39605       39605  66.465  63.67  67.845  30.30  112.60  275.52  4.33  0.16   
39606       39606  66.465  65.67  69.884  30.16  112.90  276.06  4.38  0.14   

        X_14  ...    Y_05    Y_06   Y_07    Y_08   

In [28]:
train_df, valid_df = train_test_split(df1, train_size=0.8)

train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

test_x = pd.read_csv(file_path + './test.csv').drop(columns=['ID'])

In [29]:
train_x

,X_01,X_03,X_06,X_07,X_08,X_09,X_12,X_13,X_14,X_15,...,X_44,X_45,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
38194,70.544,66.17,70.904,29.56,95.41,158.21,4.37,0.19,13.39,13.38,...,21.21,0.15,12544.13,134.682210,129.947949,139.472643,132.299986,124.762020,133.686459,134.556606
21557,67.485,65.17,69.884,29.85,105.58,117.71,4.34,0.17,13.35,13.35,...,21.14,0.15,12624.33,132.298199,138.071385,143.807214,132.119505,125.568508,138.441121,124.334309
4562,70.544,72.37,70.904,28.93,116.56,278.62,4.38,0.14,13.38,13.41,...,21.15,0.10,18394.33,125.810010,122.519528,132.610788,130.748144,120.680096,134.852993,124.379109
39428,67.485,65.57,68.864,32.07,116.43,234.22,4.37,0.17,13.39,13.40,...,21.21,0.21,0.00,129.868550,129.021214,141.274553,130.210280,128.719257,143.581967,128.091419
15695,67.485,65.17,67.845,31.44,105.68,332.51,4.38,0.18,13.41,13.38,...,21.08,0.25,16703.43,124.206837,130.821041,135.350626,125.720045,131.959202,144.584767,129.742230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,69.524,71.97,70.904,0.00,120.52,0.00,4.34,0.15,13.35,13.37,...,21.09,0.19,15585.73,124.925084,124.813091,123.314043,116.561773,119.467594,122.416775,123.261044
11284,68.504,71.67,72.943,30.90,119.81,268.32,4.38,0.09,13.40,13.42,...,21.08,0.13,14228.73,126.370267,130.973187,135.857515,133.020071,132.835186,130.027691,126.445255
38158,71.563,66.77,71.923,28.67,0.00,0.00,4.38,0.14,13.38,13.38,...,21.20,0.13,12088.23,137.139912,134.100041,147.350031,130.241667,134.185695,147.163205,133.407581
860,68.504,70.67,68.864,29.44,0.00,200.89,4.38,0.20,13.30,13.34,...,21.15,0.20,19974.53,130.648595,127.940746,135.476869,124.291716,122.022510,128.166859,124.283352


In [30]:
valid_x

,X_01,X_03,X_06,X_07,X_08,X_09,X_12,X_13,X_14,X_15,...,X_44,X_45,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
17963,66.465,62.57,67.845,28.91,101.23,260.04,4.36,0.17,13.30,13.30,...,21.14,0.12,12604.63,128.065556,142.927876,143.934663,127.675734,128.842655,134.760060,126.813557
14237,70.544,78.17,71.923,28.86,111.90,214.75,4.38,0.14,13.38,13.41,...,21.21,0.12,15321.23,127.643056,133.561871,140.044099,127.529585,126.330578,133.354809,133.543911
2992,68.504,68.17,68.864,28.14,108.46,364.07,4.34,0.15,13.36,13.39,...,21.11,0.17,12411.43,136.358098,129.140545,132.645387,126.858297,124.748920,128.367512,121.985188
18587,0.000,81.37,74.983,30.13,0.00,188.52,4.38,0.11,13.40,13.40,...,21.22,0.12,14008.93,131.659291,125.397381,148.744196,131.340460,135.328876,144.221821,130.223950
19112,65.445,64.57,67.845,27.11,103.94,223.55,4.36,0.15,13.34,13.33,...,21.14,0.16,10530.03,132.419370,129.721226,143.947633,132.284460,126.447689,146.282254,133.663632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33978,69.524,65.57,69.884,29.50,128.35,178.23,4.37,0.11,13.45,13.43,...,21.19,0.13,13561.53,137.524754,131.410183,139.238588,135.518782,124.800785,138.075589,135.456371
22572,72.583,71.17,72.943,29.66,102.68,171.40,4.38,0.14,13.38,13.41,...,21.17,0.16,13979.33,121.420750,126.243290,136.164087,122.200575,130.297443,133.995617,125.595280
2865,68.504,69.97,68.864,27.61,0.00,144.28,4.39,0.17,13.33,13.34,...,21.18,0.11,14287.43,130.814908,128.802586,135.012003,125.067930,123.093868,140.852940,118.918759
22194,67.485,61.87,67.845,29.39,112.83,202.16,4.36,0.00,13.30,13.33,...,21.15,0.12,12598.63,135.452397,131.939602,139.432458,124.437475,124.588634,135.540692,123.279461


In [31]:
train_y

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
38194,1.089,0.913,0.632,14.841,32.791,16.298,2.642,-26.795,-26.826,-22.648,24.006,-26.769,-26.823,-26.686
21557,1.364,1.032,0.834,16.024,33.842,16.879,3.374,-26.348,-26.455,-22.620,24.610,-26.288,-26.334,-26.197
4562,0.889,0.708,0.424,16.660,33.291,17.034,2.964,-25.591,-25.341,-21.918,25.102,-25.521,-25.537,-25.384
39428,0.803,0.439,0.569,10.683,29.167,15.131,3.493,-27.566,-27.269,-24.639,22.703,-27.372,-27.240,-27.097
15695,1.453,0.925,1.006,10.634,29.238,17.182,3.692,-25.691,-25.587,-21.734,24.775,-25.572,-25.372,-25.539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1.312,0.757,1.041,17.950,31.016,16.009,3.256,-26.671,-26.792,-22.848,23.879,-26.639,-26.588,-26.715
11284,1.368,1.049,1.011,12.737,29.152,16.306,3.178,-26.242,-26.176,-22.312,24.066,-26.156,-26.039,-26.176
38158,1.735,1.269,1.186,17.192,33.547,17.763,2.623,-25.821,-26.011,-21.159,25.426,-25.953,-26.116,-25.805
860,1.730,1.363,1.437,18.353,31.019,16.840,3.117,-25.615,-25.326,-21.557,24.704,-25.407,-25.450,-25.228


In [32]:
valid_y

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
17963,1.326,0.824,0.783,13.284,31.265,17.226,3.664,-25.720,-25.642,-22.282,25.073,-25.509,-25.683,-25.470
14237,1.069,0.737,0.660,13.152,31.978,17.011,3.168,-25.963,-25.848,-21.769,24.791,-26.042,-25.954,-25.801
2992,1.278,1.146,0.974,13.604,32.053,16.582,3.153,-26.120,-25.842,-22.228,24.366,-26.071,-25.970,-26.009
18587,1.476,1.037,1.049,15.259,32.896,17.680,2.976,-25.818,-25.606,-21.670,25.364,-25.684,-25.707,-25.563
19112,1.428,1.083,1.038,17.379,32.569,16.785,3.079,-25.617,-25.460,-21.871,24.694,-25.479,-25.464,-25.652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33978,1.374,1.051,0.709,17.020,34.488,16.682,3.248,-26.362,-26.338,-22.245,24.146,-26.241,-26.499,-26.277
22572,1.376,0.907,1.118,10.528,29.808,15.943,3.192,-27.136,-27.555,-23.336,23.436,-27.023,-26.916,-27.015
2865,1.720,1.155,1.124,15.023,29.330,17.696,2.952,-25.137,-25.199,-21.047,25.431,-25.254,-25.171,-24.996
22194,1.212,0.731,0.509,15.485,30.060,17.004,3.547,-26.120,-26.015,-22.567,24.655,-25.811,-25.825,-25.986


In [33]:
LR = MultiOutputRegressor(LinearRegression()).fit(train_x, train_y)

valid_preds = LR.predict(valid_x)
lg_nrmse(valid_y.values, valid_preds)

(1.9902036440743438,
 [0.2623521461414246,
  0.3622442903597561,
  0.35504227248292225,
  0.19338551069635598,
  0.08045773129177997,
  0.10553946015093879,
  0.13096754765112098,
  0.024789719310051375,
  0.0246368724375617,
  0.0398096209128281,
  0.033935798544786226,
  0.024768350974868212,
  0.024656624332159538,
  0.024661963170919836])